In [ ]:
%matplotlib inline

import datetime as dt
import itertools as it

from IPython.core.interactiveshell import InteractiveShell
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import synapseclient
from synapseclient import Activity, Schema, Table, as_table_columns

InteractiveShell.ast_node_interactivity = 'all'
syn = synapseclient.Synapse()
syn.login()

def isnum(x):
    if x is None:
        return False
    try:
        float(x)
        return True
    except ValueError:
        return False

def safe_round(x):
    if not pd.isnull(x) and isnum(x):
        return np.round(x, 3)
    else:
        return x

### Prepare V1 data

In [ ]:
v1sid = 'syn10236538'
v1r = pd.read_csv(syn.get(v1sid).path, sep='\t')
v1r.head()

In [ ]:
v1 = v1r.drop(columns=[
    'user_id', 'study_arm', 'start'
]).rename(columns={
    'brightenid':'participant_id',
    'passive_date_pacific':'dt_passive'
})

v1.columns = [c.lower() for c in v1.columns]

v1.head()

In [ ]:
toround = list(set(v1.columns) - {'dt_response', 'user_phone_type', 'cohort', 'participant_id', 'day', 'week'})
for c in toround:
    v1[c] =v1[c].apply(safe_round)

v1.head()

In [ ]:
colorder = ['participant_id', 'dt_passive', 'day', 'week', 'cohort', 'user_phone_type']
colorder = colorder + sorted(list(set(v1.columns) - set(colorder)))
v1 = v1[colorder]

v1.head()

In [ ]:
v1.mobility.hist()
v1.mobility.describe()

In [ ]:
'an element $$\(\in\)$$ {' + ', '.join([f'_{c}_' for c in v1.user_phone_type.unique() if str(c) != 'nan']) + '}'

In [ ]:
final = syn.store(Table(
    Schema(
            name='V1 Passive Features',
            columns=as_table_columns(v1), 
            parent='syn10848316'),
        v1
    )
)

In [ ]:
final = syn.setProvenance(
    'syn17025494',
    activity=Activity(
        name='Generate V1 Table Data',
        description='Process the data collected during study ',
        used=[v1sid],
        executed=[
            dict(
                name='IPython Notebook',
                url='https://github.com/apratap/BRIGHTEN-Data-Release/blob/master/Create_V1PassiveFeatures_datafiles.ipynb'
            )
        ]
    )
)